In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

dir = "/content/drive/MyDrive/ColabNotebooks/BTdsYesNo/"

# dataset_path = os.listdir(dir)

tumor_types = os.listdir(dir)

print (tumor_types)  #Tumor this dataset

# print("Types ", len(dataset_path))

Instructions for updating:
non-resource variables are not supported in the long term


['no', 'yes']


In [ ]:
tumors = []

for item in tumor_types:
 # Get all the file names
 all_tumor= os.listdir(dir + '/' +item)
 #print(all_tumor)

 # Add them to the list
 for tumor in all_tumor:
    tumors.append((item, dir + '/' + tumor))

print(tumors)

[('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no266.jpg'), ('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no1410.jpg'), ('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no175.jpg'), ('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no150.jpg'), ('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no183.jpg'), ('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no234.jpg'), ('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no1164.jpg'), ('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no1135.jpg'), ('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no1286.jpg'), ('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no1194.jpg'), ('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no1482.jpg'), ('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no1281.jpg'), ('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no163.jpg'), ('no', '/content/drive/MyDrive/ColabNotebooks/BTdsYesNo//no1070.jpg'), ('no', '/co

In [ ]:
# Build a dataframe
tumors_df = pd.DataFrame(data=tumors, columns=['tumor type', 'image'])
print(tumors_df.head())
print(tumors_df.tail())

  tumor type                                              image
0         no  /content/drive/MyDrive/ColabNotebooks/BTdsYesN...
1         no  /content/drive/MyDrive/ColabNotebooks/BTdsYesN...
2         no  /content/drive/MyDrive/ColabNotebooks/BTdsYesN...
3         no  /content/drive/MyDrive/ColabNotebooks/BTdsYesN...
4         no  /content/drive/MyDrive/ColabNotebooks/BTdsYesN...
     tumor type                                              image
2995        yes  /content/drive/MyDrive/ColabNotebooks/BTdsYesN...
2996        yes  /content/drive/MyDrive/ColabNotebooks/BTdsYesN...
2997        yes  /content/drive/MyDrive/ColabNotebooks/BTdsYesN...
2998        yes  /content/drive/MyDrive/ColabNotebooks/BTdsYesN...
2999        yes  /content/drive/MyDrive/ColabNotebooks/BTdsYesN...


In [ ]:
# Let's check how many samples for each category are present
print("Total number of tumors in the dataset: ", len(tumors_df))


Total number of tumors in the dataset:  3000


In [ ]:
tumor_count = tumors_df['tumor type'].value_counts()

print("tumors in each category: ")
print(tumor_count)


tumors in each category: 
no     1500
yes    1500
Name: tumor type, dtype: int64


In [ ]:
import cv2

img_dir = dir
no_tumor = dir + '/no'

im_size = 60
images = []
labels = []
dataset = []

# compiling all the images including yes and no tumor images into list images and labels
for i in tumor_types:
    data_path = img_dir + str(i)  # entered in 1st folder and then 2nd folder and then 3rd folder
    filenames = [i for i in os.listdir(data_path) if i.endswith('.jpg')]
    # print(filenames)
    # print( img_dir + i + '/' + filenames[0])
    # will get the names of all images which ends with .jpg extension
    for f in filenames:
        try:
            img = cv2.imread(img_dir + i + '/' + f)  # reading that image as array
        #print(img)  # will get the image as an array
            img = cv2.resize(img, (im_size, im_size), interpolation=cv2.INTER_AREA)
            images.append(img)
            labels.append(i)
        except:
            print("Error occuring at " + f)



3000
3000


In [ ]:
print(len(labels) )
print(len(images))

3000
3000


In [ ]:
# Transform the image array to a numpy type

images = np.array(images)

images.shape

(3000, 60, 60, 3)

In [ ]:
images = images.astype('float32') / 255.0

In [ ]:
images.shape

(3000, 60, 60, 3)

In [ ]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder


y=tumors_df['tumor type'].values
print(y[:5])
print(y[-5:])

['no' 'no' 'no' 'no' 'no']
['yes' 'yes' 'yes' 'yes' 'yes']


In [ ]:
# for y
y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

[0 0 0 ... 1 1 1]


In [ ]:
Y=y.reshape(-1,1)
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [0])],remainder='passthrough')
Y=np.array(columnTransformer.fit_transform(Y),dtype=np.str)


#onehotencoder = OneHotEncoder(categorical_features=[0])  #Converted  scalar output into vector output where the correct class will be 1 and other will be 0
#Y= onehotencoder.fit_transform(y)
Y.shape

<ipython-input-22-675aa920a0d0>:5: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y=np.array(columnTransformer.fit_transform(Y),dtype=np.str)


(3000, 2)

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.2, random_state=415)

In [ ]:
#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(2400, 60, 60, 3)
(2400, 2)
(600, 60, 60, 3)
(600, 2)


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# Number of classes, one class for each of tumor.
num_classes = 2

# flattened imge
n_input = 10800


# architecture hyper-parameter
learning_rate = 0.001
training_iters = 10
batch_size = 16
display_step = 20

In [ ]:
img_size=60
num_channels=3
#x = tf.placeholder(tf.float32, [None, n_input])
x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels])  # None,60,60,3

y_ = tf.placeholder(tf.float32, [None, num_classes]) # None,3



print('Shape of placeholder',x.shape, y_.shape)

Shape of placeholder (?, 60, 60, 3) (?, 2)


In [ ]:
# optional use- if you want to add bias in conv layer then use this function by calling it in conv_net model
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')   # summation = wx+b, Activation
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [ ]:
weights = {
    'w1': tf.Variable(tf.random_normal([5, 5, 3, 32]),name='w1'),
    'w2': tf.Variable(tf.random_normal([5, 5, 32, 64]),name='w2'),
    'w3': tf.Variable(tf.random_normal([5, 5, 64, 128]),name='w3'),
    'wd1': tf.Variable(tf.random_normal([8 * 8 * 128, 2048]),name='wd1'),
    'wout': tf.Variable(tf.random_normal([2048, num_classes]),name='wout')
}

biases = {
    'b1': tf.Variable(tf.random_normal([32]),name='b1'),
    'b2': tf.Variable(tf.random_normal([64]),name='b2'),
    'b3': tf.Variable(tf.random_normal([128]),name='b3'),
    'bd1': tf.Variable(tf.random_normal([2048]),name='bd1'),
    'bout': tf.Variable(tf.random_normal([num_classes]),name='bout')
}

In [ ]:
def conv_net(x, weights, biases):

    # reshape input to 60x60x3 size
    x = tf.reshape(x, shape=[-1, 60, 60, 3])

    print("###########################################################################")
    print("size of x is")
    print(x.shape)


    conv1 = conv2d(x, weights['w1'], biases['b1'])
    conv1 = maxpool2d(conv1, k=2)
    print("###########################################################################")
    print("size after 1st conv layer is ")
    print(conv1.shape)


    #input is 30*30*32 image
    # Convolution Layer
    conv2 = conv2d(conv1, weights['w2'], biases['b2'])
    conv2 = maxpool2d(conv2, k=2)
    print("###########################################################################")
    print("size after 2nd conv and pooling layer is")
    print(conv2.shape)


    ### third conv layer
    # input is 15*15*64 image
    # Convolution Layer
    conv3 = conv2d(conv2, weights['w3'], biases['b3'])

    conv3 = maxpool2d(conv3, k=2)
    print("###########################################################################")
    print("size after 3rd conv and pooling layer is")
    print(conv3.shape)

     #input is 8*8*128

    # Fully connected layer
    # Reshape conv3 output to fit fully connected layer input   = 8*8*128 = 8192
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    print("###########################################################################")
    print("shape after flattening the image")
    print(fc1)  #8192 is the output


    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    print("###########################################################################")
    print("shape after fully connected layer")
    print(fc1)


    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term.
    out = tf.add(tf.matmul(fc1, weights['wout']), biases['bout'])
    print("###########################################################################")
    print("Output layer")
    return out

In [ ]:
# Create the model
model = conv_net(x, weights, biases)
print(model)



###########################################################################
size of x is
(?, 60, 60, 3)
###########################################################################
size after 1st conv layer is 
(?, 30, 30, 32)
###########################################################################
size after 2nd conv and pooling layer is
(?, 15, 15, 64)
###########################################################################
size after 3rd conv and pooling layer is
(?, 8, 8, 128)
###########################################################################
shape after flattening the image
Tensor("Reshape_1:0", shape=(?, 8192), dtype=float32)
###########################################################################
shape after fully connected layer
Tensor("Relu_3:0", shape=(?, 2048), dtype=float32)
###########################################################################
Output layer
Tensor("Add_1:0", shape=(?, 2), dtype=float32)


In [ ]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y_))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
cost_history=[]
n_epochs =25
# the execution
sess = tf.Session()
sess.run(init)

#train_y=train_y.toarray()

for i in range(n_epochs):
    a, c = sess.run([optimizer, cost], feed_dict={x: train_x, y_: train_y})  #working
    cost_history = np.append(cost_history,c)  # working
    print('epoch : ', i,  ' - ', 'cost: ', c) #worki

epoch :  0  -  cost:  3818041.2
epoch :  1  -  cost:  3158348.2
epoch :  2  -  cost:  3070346.8
epoch :  3  -  cost:  968740.25
epoch :  4  -  cost:  1503665.9
epoch :  5  -  cost:  2190463.2
epoch :  6  -  cost:  1484663.9
epoch :  7  -  cost:  769087.44
epoch :  8  -  cost:  973316.56
epoch :  9  -  cost:  1408839.4
epoch :  10  -  cost:  1428872.0
epoch :  11  -  cost:  1093775.5
epoch :  12  -  cost:  755134.2
epoch :  13  -  cost:  697743.94
epoch :  14  -  cost:  913290.25
epoch :  15  -  cost:  996130.0
epoch :  16  -  cost:  796901.3
epoch :  17  -  cost:  519855.38
epoch :  18  -  cost:  484794.25
epoch :  19  -  cost:  600653.0
epoch :  20  -  cost:  629656.06
epoch :  21  -  cost:  489160.3
epoch :  22  -  cost:  308711.53
epoch :  23  -  cost:  340471.25
epoch :  24  -  cost:  478248.06


In [ ]:
#test_y=test_y.todense()  #working solution of ValueError: setting an array element with a sequence.
#print(test_y)

correct_prediction = tf.equal(tf.argmax(model,1), tf.argmax(y_,1))
correct_prediction

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy


# retrun the accuracy on the test set.
print("Accuracy: ", sess.run(accuracy, feed_dict={x: test_x, y_:test_y}))

Accuracy:  0.72333336
